In [ ]:
import cv2
import numpy as np
from os import listdir
import operator
from google.colab.patches import cv2_imshow
matchCount={}
light_purple =(120,70,70)
dark_purple =(170,255,255)
cntR,cntT,cntS,cntG=[],[],[],[]
count={'ring':0,'troph':0,'schizont':0,'gametocyte':0}
#acc={'ring':0,'troph':0,'schizont':0,'gametocyte':0}
search_keys=['ring','troph','schizont','gametocyte']

for filenames in listdir('/content/drive/My Drive/Final Year Project/Test Images'):
  img1 = cv2.imread("/content/drive/My Drive/Final Year Project/Test Images/"+filenames)
  for sub in search_keys:
      if(filenames.find(sub))!=-1:
          if sub in count:
              count[sub] += 1
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
  hsv_img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2HSV)

  mask = cv2.inRange(hsv_img1, light_purple, dark_purple)
  res1 = cv2.bitwise_and(img1, img1, mask=mask)
  print(filenames)
  for filename in listdir('/content/drive/My Drive/Final Year Project/Sample Images'):
      img2 = cv2.imread("/content/drive/My Drive/Final Year Project/Sample Images/"+filename, cv2.IMREAD_GRAYSCALE+cv2.THRESH_OTSU)
      # ORB Detector
      img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
      hsv_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
      mask = cv2.inRange(hsv_img2, light_purple, dark_purple)
      res2 = cv2.bitwise_and(img2, img2, mask=mask)
      orb = cv2.ORB_create()
      kp1, des1 = orb.detectAndCompute(res1, None)
      kp2, des2 = orb.detectAndCompute(res2, None)
      # Brute Force Matching
      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
      matches = bf.match(des1, des2)
      matches = [m for m in matches if m.distance<60]
      #print(len(matches))
      matchCount[filename]=len(matches)
      matching_result = cv2.drawMatches(res1, kp1, res2, kp2, matches, None, flags=2)
      #cv2.imshow("Img1", img1)
      #cv2.imshow("Img2", img2)
      #cv2.imshow("Matching result", matching_result)
      #cv2.waitKey(0)
      #cv2.destroyAllWindows()
  #print(matchCount)
  sorted_d = dict(sorted(matchCount.items(), key=operator.itemgetter(1),reverse=True))
  classify={}
  acc={}
  classify.clear()
  first5pairs = {k: sorted_d[k] for k in list(sorted_d)[:5]}
  first5=dict(first5pairs)

  #print(list(sorted_d.keys())[:5])
  print(first5pairs)

  for sKey in search_keys:
      if sKey=='ring':
          ring = dict(filter(lambda item: sKey in item[0], first5.items()))
          try:
              classify['ring']=sum(ring.values())
          except:
              print("ring0")
      elif sKey=='troph':
          troph = dict(filter(lambda item: sKey in item[0], first5.items()))
          try:
              classify['troph']=sum(troph.values())
          except:
              print("troph0")
      elif sKey=='schizont':
          schizont = dict(filter(lambda item: sKey in item[0], first5.items()))
          try:
              classify['schizont']=sum(schizont.values())
          except:
              print("schizont0")
      elif sKey=='gametocyte':
          gametocyte = dict(filter(lambda item: sKey in item[0], first5.items()))
          try:
              classify['gametocyte']=sum(gametocyte.values())
          except:
              print("gametocyte0")
  #print(classify)
  classified = max(classify, key=classify.get)
  #print(classified+" == "+filenames)
  if (filenames.find(classified) != -1):
      print (filenames+" classified accurately as",classified)

      if classified=='ring': cntR.append(1)
      elif classified=='troph': cntT.append(1)
      elif classified=='schizont': cntS.append(1)
      elif classified=='gametocyte': cntG.append(1)

  else:
      print (filenames+" classified wrongly as",classified)

  print('\n')

totalMatch=sum(cntR)+sum(cntT)+sum(cntS)+sum(cntG)
totalTest=sum(count.values())
print('totalMatch =',totalMatch)
print('totalTest =',totalTest)
print('Accuracy=',(totalMatch/totalTest)*100)